In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import asyncio
import redis
import numpy as np

import confirm.cloud.clickhouse as ch

from confirm.adagrid.validation import AdaValidate
import confirm.adagrid.adagrid as adagrid
import imprint as ip
from imprint.models.ztest import ZTest1D

import clickhouse_connect
clickhouse_connect.common.set_setting('autogenerate_session_id', False)

In [2]:
def random_assignments(seed, n_tiles, n_workers, names):
    # Randomly assign tiles to packets.
    np.random.seed(seed)
    splits = np.array_split(np.arange(n_tiles), n_workers)
    assignment = np.empty(n_tiles, dtype=np.int32)
    for i in range(n_workers):
        assignment[splits[i]] = names[i]
    rng = np.random.default_rng()
    rng.shuffle(assignment)
    return assignment

### Glossary

- **coordination**: All workers come together and divide up all tiles.
- **step**: A worker checks the convergence criterion, select tiles, and simulates.
- **packet**: The unit of simulation work that a worker requests from the DB.
- **batch**: The unit of simulation work that is passed to a Model.

In [68]:
# glossary

db = ch.Clickhouse.connect()
g = ip.cartesian_grid(
    theta_min=[-1], theta_max=[1], n=[50000], null_hypos=[ip.hypo("x0 < 0")]
)
kwargs = dict(
    lam=-1.96,
    model_seed=0,
    model_kwargs=None,
    delta=0.01,
    init_K=2**13,
    n_K_double=4,
    tile_batch_size=64,
    max_target=0.002,
    global_target=0.005,
    n_steps = 100,
    step_size=2**10,
    n_iter=1000,
    packet_size = None,
    prod = True,
    overrides = None,
    callback=adagrid.print_report,
    backend=adagrid.LocalBackend(),
    
    model_type=ZTest1D
)
assignment_seed = 1
n_workers = 2

redis_con = db.redis_con
job_id = db.job_id
initial_worker_ids = [redis_con.incr(f"{job_id}:worker_id") + 1 for _ in range(n_workers)]

# TODO: the assignment_seed should change with each coordination.
g.df['worker_assignment'] = random_assignments(assignment_seed, g.n_tiles, n_workers, initial_worker_ids)

packet_size = 2000
import pandas as pd
def assign_packets(df):
    return pd.Series(np.floor(np.arange(df.shape[0]) / packet_size).astype(int), df.index)
g.df['packet_id'] = g.df.groupby('worker_assignment')['worker_assignment'].transform(assign_packets)

ada = adagrid.Adagrid(ZTest1D, g, db, AdaValidate, print, None, kwargs, worker_id = 1)

INFO:confirm.cloud.clickhouse:[worker_id=1] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=1] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/14e0a625ccff46c68a11992f31f08558
INFO:confirm.cloud.clickhouse:[worker_id=1] 
Connected to job 14e0a625ccff46c68a11992f31f08558
DEBUG:imprint.grid:[worker_id=1] 
_gen_short_uuids(n=50000, worker_id=1, t=1676050756, n_bits=18, worker_bits=18) = [4496866493040164864 4496866493040164865 4496866493040164866, ...]:
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set_or_append(config) -> False None
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(config) -> False config
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
writing 25000 tiles
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set step info: (-1, 0, 0, 0)
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(null_hypos) -> False null_hypos
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
first step (0, 0, 25,

In [69]:
db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,worker_assignment,packet_id,K,step_id,step_iter,creator_id,creation_time,active
0,4496866493040164876,0,-0.99950,0.00002,1,2,0,8192,0,0,1,1.676051e+09,1
1,4496866493040164891,0,-0.99890,0.00002,1,2,0,8192,0,0,1,1.676051e+09,1
2,4496866493040164894,0,-0.99878,0.00002,1,3,0,8192,0,0,1,1.676051e+09,1
3,4496866493040164912,0,-0.99806,0.00002,1,3,0,8192,0,0,1,1.676051e+09,1
4,4496866493040164970,0,-0.99574,0.00002,1,2,0,8192,0,0,1,1.676051e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,4496866493040189773,0,-0.00362,0.00002,1,2,6,8192,0,24,1,1.676051e+09,1
24996,4496866493040189787,0,-0.00306,0.00002,1,3,6,8192,0,24,1,1.676051e+09,1
24997,4496866493040189812,0,-0.00206,0.00002,1,3,6,8192,0,24,1,1.676051e+09,1
24998,4496866493040189826,0,-0.00150,0.00002,1,3,6,8192,0,24,1,1.676051e+09,1


DEBUG:confirm.adagrid:[worker_id=1] 
Acquired lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2
DEBUG:confirm.adagrid:[worker_id=1] 
Added worker 2 to workers set.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.adagrid:[worker_id=1] 
Removed worker 2 from workers set.
DEBUG:confirm.adagrid:[worker_id=1] 
Successfully released lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2.


In [71]:
class NewAdagrid:
    def __init__(self, worker_id):
        self.first_insert = True
        self.worker_id = worker_id

    async def process_step(self, step_id):
        next_work = None
        packet_id = 0
        insert_threads = []
        packet_id = 0
        while True:
            report = dict()
            report['worker_id'] = self.worker_id
            report['step_id'] = step_id
            report['packet_id'] = packet_id

            ########################################
            # Get work
            ########################################
            def get_work():
                return ch._query_df(
                    db.client,
                    f"""
                    select * from tiles
                        where
                            worker_assignment = {self.worker_id}
                            and step_id = {step_id}
                            and packet_id = {packet_id}
                    """,
                )

            start = time.time()
            # On the first loop, we won't have queued any work queries yet.
            if next_work is None:
                next_work = asyncio.to_thread(get_work)
            work = await next_work
            report['n_tiles'] = work.shape[0]

            # Empty packet is an indication that we are done with this step.
            if work.shape[0] == 0:
                report['status'] = 'PACKET_DONE'
                insert_threads.append(asyncio.to_thread(db.insert_report, report))
                report['runtime_done_wait'] = time.time() - start
                await asyncio.gather(*insert_threads)
                break

            # Queue a query for the next packet.
            next_work = asyncio.to_thread(get_work)

            # Check if some other worker has already inserted this packet.
            packet_flag = f"{job_id}:worker_{self.worker_id}_step_{step_id}_packet_{packet_id}_insert"
            flag = redis_con.get(packet_flag)
            if flag is not None:
                logger.debug(
                    "Skipping packet. Flag "
                    f"{packet_flag} is set by worker_id={flag.decode('ascii')}."
                )
                packet_id += 1
                report["runtime_skip_packet"] = time.time() - start
                report["status"] = 'SKIPPED'
                insert_threads.append(asyncio.to_thread(db.insert_report, report))
                continue
            report["runtime_get_work"] = time.time() - start

            ########################################
            # Process tiles
            ########################################
            start = time.time()
            results_df = ada.algo.process_tiles(tiles_df=work, report=report)
            report["runtime_process_tiles"] = time.time() - start

            ########################################
            # Insert results
            ########################################
            start = time.time()
            was_flag_set = redis_con.setnx(packet_flag, self.worker_id)
            if was_flag_set == 0:
                logger.warning(
                    f"(step_id={step_id}, packet_id={packet_id})"
                    " already inserted, discarding results."
                )
                report['status'] = 'DISCARDED'
            else:
                if self.first_insert:
                    # Do the first insert synchronously to make sure the results table is created.
                    db.insert_results(results_df, "total_cost_order, tie_bound_order")
                    self.first_insert = False
                else:
                    insert_threads.append(
                        asyncio.to_thread(
                            db.insert_results,
                            results_df,
                            "total_cost_order, tie_bound_order",
                        )
                    )
                logger.debug(
                    "inserted packet results for "
                    f"(step_id = {step_id}, packet_id={packet_id})"
                    f" with {results_df.shape[0]} results"
                )
                report['status'] = 'WORK'
            insert_threads.append(asyncio.to_thread(db.insert_report, report))
            report["runtime_insert_results"] = time.time() - start
            packet_id += 1

In [72]:
import imprint.log
worker_id = initial_worker_ids[0]
imprint.log.worker_id.set(worker_id)

ada_new = NewAdagrid(worker_id)
async with HeartbeatThread(worker_id):
    await ada_new.process_step(0)
    print("finished step")

DEBUG:confirm.adagrid:[worker_id=2] 
Acquired lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2
DEBUG:confirm.adagrid:[worker_id=2] 
Added worker 2 to workers set.
DEBUG:confirm.adagrid:[worker_id=2] 
Extended lock 14e0a625ccff46c68a11992f31f08558:heartbeat_lock_2 for 30 seconds.
DEBUG:confirm.cloud.clickhouse:[worker_id=2] 
writing 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=0) with 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=1) with 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=2) with 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=3) with 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=4) with 2000 results
DEBUG:confirm.adagrid:[worker_id=2] 
inserted packet results for (step_id = 0, packet_id=5) with 

finished step


In [77]:
def worst_tile(db, worker_id, order_col):
    return ch._query_df(
        db.client,
        f"""
        select * from results r
            where
                active=true
                and id not in (select id from inactive)
                and worker_assignment = {worker_id}
        order by {order_col} limit 1
    """,
    )
# db.n_processed_tiles(0)
# ada.algo.convergence_criterion

In [ ]:
def new_step(self, tiles_df, new_step_id, report):
    tiles_df["finisher_id"] = self.cfg["worker_id"]
    tiles_df["active"] = ~(tiles_df["refine"] | tiles_df["deepen"])

    # Record what we decided to do.
    if "split" not in tiles_df.columns:
        tiles_df["split"] = False
    done_cols = [
        "id",
        "step_id",
        "step_iter",
        "active",
        "finisher_id",
        "refine",
        "deepen",
        "split",
    ]
    # TODO: call finish in separate thread to avoid blocking
    self.db.finish(tiles_df[done_cols])

    n_refine = tiles_df["refine"].sum()
    n_deepen = tiles_df["deepen"].sum()
    report.update(
        dict(
            n_refine=n_refine,
            n_deepen=n_deepen,
            n_complete=tiles_df["active"].sum(),
        )
    )

    nothing_to_do = n_refine == 0 and n_deepen == 0
    if nothing_to_do:
        return "empty"

    # Actually deepen and refine!
    g = refine_and_deepen(
        tiles_df, self.null_hypos, self.cfg["max_K"], self.cfg["worker_id"]
    )
    g.df["step_id"] = new_step_id
    g.df["creator_id"] = self.cfg["worker_id"]
    g.df["creation_time"] = imprint.timer.simple_timer()

    # there might be new inactive tiles that resulted from splitting with
    # the null hypotheses. we need to mark these tiles as finished.
    # TODO: inactive insert_tiles and finish can be done in separate thread
    # to avoid blocking
    inactive_df = g.df[~g.df["active"]].copy()
    inactive_df["step_iter"] = np.int32(-1)
    self.db.insert_tiles(inactive_df)
    inactive_df["refine"] = False
    inactive_df["deepen"] = False
    inactive_df["split"] = True
    inactive_df["finisher_id"] = self.cfg["worker_id"]
    self.db.finish(inactive_df[done_cols])

    # Assign tiles to packets and then insert them into the database for
    # processing.
    g_active = g.prune_inactive()
    g_active.df["step_iter"], n_packets = step_iter_assignments(
        g_active.df, self.cfg["packet_size"]
    )
    # TODO: i think insert_tiles can be done in separate thread to avoid
    # blocking
    self.db.insert_tiles(g_active.df)
    self.db.set_step_info(
        step_id=new_step_id, step_iter=0, n_iter=n_packets, n_tiles=g_active.n_tiles
    )

    logger.debug(
        f"new step {(new_step_id, 0, n_packets, g.n_tiles)}\n"
        f"n_tiles={g_active.n_tiles} packet_size={self.cfg['packet_size']}\n"
        f"n_inactive_tiles={inactive_df.shape[0]}"
    )
    report.update(
        dict(
            n_new_tiles=g.n_tiles,
            new_K_distribution=g.df["K"].value_counts().to_dict(),
        )
    )
    return new_step_id

In [89]:
# TODO: if n_processed_tiles == step_n_tiles:
report = dict()
step_id = 0
ada.algo.c["worker_id"] = worker_id
start = time.time()
converged, convergence_data = ada.algo.convergence_criterion(report)
report["runtime_convergence_criterion"] = time.time() - start
if converged:
    report["status"] = "CONVERGED"
    logger.debug("Convergence!!")
    # TODO: return
elif step_id >= ada.algo.c["n_steps"] - 1:
    report["status"] = "MAX_STEPS"
    print("max steps")
    # TODO: end the job.
else:
    # If we haven't converged, we create a new step.
    start = time.time()
    new_step_id = step_id + 1
    tiles_df = ada.algo.select_tiles(report, convergence_data)
    report["runtime_select_tiles"] = time.time() - start

    if tiles_df is None:
        # New step is empty so we have terminated but
        # failed to converge.
        logger.debug(
            "New packet is empty. Waiting for the next "
            "coordination despite failure to converge."
        )
        report['status'] = 'EMPTY_STEP'
        # TODO: wait for coordination
    else:
        report["status"] = "NEW_STEP"
        report["n_tiles"] = tiles_df.shape[0]


{'converged': False,
 'max_tie_est': 0.02355957,
 'next_tile_tie_est': 0.0235595703125,
 'next_tile_tie_bound': 0.027757570147514343,
 'next_tile_sim_cost': 0.004196513078392677,
 'next_tile_grid_cost': 1.4867566216665573e-06,
 'next_tile_total_cost': 0.004197999835014343,
 'next_tile_K': 8192.0,
 'next_tile_at_max_K': False}

In [79]:
report = dict()
max_tie_est = worst_tile(db, worker_id, "tie_est desc")["tie_est"].iloc[0]
next_tile = worst_tile(db, worker_id, "total_cost_order, tie_bound_order").iloc[0]
report['converged'] = ada.algo._are_tiles_done(next_tile, max_tie_est)
report.update(
    dict(
        max_tie_est=max_tie_est,
        next_tile_tie_est=next_tile["tie_est"],
        next_tile_tie_bound=next_tile["tie_bound"],
        next_tile_sim_cost=next_tile["sim_cost"],
        next_tile_grid_cost=next_tile["grid_cost"],
        next_tile_total_cost=next_tile["total_cost"],
        next_tile_K=next_tile["K"],
        next_tile_at_max_K=next_tile["K"] == ada.algo.max_K,
    )
)
report['converged'], max_tie_est

(False, 0.02355957)

In [82]:
report

{'converged': False,
 'max_tie_est': 0.02355957,
 'next_tile_tie_est': 0.0235595703125,
 'next_tile_tie_bound': 0.027757570147514343,
 'next_tile_sim_cost': 0.004196513078392677,
 'next_tile_grid_cost': 1.4867566216665573e-06,
 'next_tile_total_cost': 0.004197999835014343,
 'next_tile_K': 8192.0,
 'next_tile_at_max_K': False}

In [73]:
db.get_reports()

,worker_id,step_id,packet_id,n_tiles,runtime_get_work,runtime_process_tiles,status,runtime_insert_results,runtime_done_wait
0,2,0,0,2000,0.391095,0.225258,WORK,0.915440,NaN
1,2,0,3,2000,0.096891,0.023989,WORK,0.037790,NaN
2,2,0,5,2000,0.096089,0.020335,WORK,0.041483,NaN
3,2,0,2,2000,0.095760,0.019291,WORK,0.041628,NaN
4,2,0,4,2000,0.095944,0.023964,WORK,0.037034,NaN
5,2,0,6,500,0.066075,0.104144,WORK,0.036042,NaN
6,2,0,1,2000,0.157111,0.022715,WORK,0.038953,NaN
7,2,0,7,0,NaN,NaN,PACKET_DONE,NaN,0.033856
